In [1]:
%matplotlib ipympl

In [2]:
import pylab as pyl
import pygeode as pyg
from pygeode import timeutils as tu
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime
import h5py
import scipy.io
import scipy.signal
import scipy.stats

In [3]:
import plt_helpers as plth

In [4]:
f = h5py.File('/data/athena/adk33/dotmat/erai_divF_daily_all_1979_2016.mat','r')
eraiT = np.array(f.get('T')) 
eraiu = np.array(f.get('u'))
erailat = np.array(f.get('lat')) 
eraip = np.array(f.get('p')) 
f.close()

In [5]:
erai_pres = pyg.Pres(eraip[0,:]/100.0)
erai_time =pyg.ModelTime365(values=np.arange(38*365), units = 'days', startdate=dict(year=1979, month=1, day=1))
erai_lat = pyg.Lat(erailat[0,:]*180.0/np.pi)

In [6]:
erai_T = pyg.Var([erai_time, erai_pres, erai_lat], name='erai T', values=np.reshape(eraiT, [len(erai_time), len(erai_pres), len(erai_lat)]))
erai_u = pyg.Var([erai_time, erai_pres, erai_lat], name='erai u', values=np.reshape(eraiu, [len(erai_time), len(erai_pres), len(erai_lat)]))

In [51]:
pyg.save('erai_u_T.nc', pyg.Dataset([erai_T(pres=(67,96)),erai_u(pres=(67,96))]), format='netcdf4')

/home/adk33/aether/share/minconda3/envs/pyg3/lib/python3.6/site-packages/pygeode-1.3.0rc2-py3.6-linux-x86_64.egg/pygeode/formats/cfmeta.py:136: UserWarning: renaming 'erai T' to 'erai_T'
  warn ("renaming '%s' to '%s'"%(oldname,newname))
/home/adk33/aether/share/minconda3/envs/pyg3/lib/python3.6/site-packages/pygeode-1.3.0rc2-py3.6-linux-x86_64.egg/pygeode/formats/cfmeta.py:136: UserWarning: renaming 'erai u' to 'erai_u'
  warn ("renaming '%s' to '%s'"%(oldname,newname))
/home/adk33/aether/share/minconda3/envs/pyg3/lib/python3.6/site-packages/pygeode-1.3.0rc2-py3.6-linux-x86_64.egg/pygeode/formats/netcdf4.py:112: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  v = ncfile.createVariable(var.name, datatype=var.dtype, dimensions=dimensions, zlib=compress, fill_value=var.atts.get('_FillValue',None))


In [52]:
erai = pyg.open('erai_u_T.nc')
era

<Dataset>:
Vars:
  erai_T (time,pres,lat)  (13870,2,179)
  erai_u (time,pres,lat)  (13870,2,179)
Axes:
  time <ModelTime365>:  Jan 1, 1979 00:00:00 to Dec 31, 2016 00:00:00 (13870 values)
  pres <Pres>    :  80 hPa to 96 hPa (2 values)
  lat <Lat>      :  89 N to 89 S (179 values)
Global Attributes:
  {}


In [7]:
du_dz = erai_u.deriv(erai_u.pres, dx=pyg.log(erai_pres))

In [11]:
dT = erai_T - pyg.climatology(erai_T)
T_series = dT.squeeze()
du_dz_series = du_dz.squeeze()
#T_du_dz_corr = pyg.regress(T_series, du_dz_series, axes=[T_series.time,], output='m')

/home/adk33/aether/share/minconda3/envs/pyg3/lib/python3.6/site-packages/pygeode-1.3.0rc2-py3.6-linux-x86_64.egg/pygeode/timeutils.py:20: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  in_atts = np.vstack(f[S] for f in fields).transpose()


In [12]:
slope_T_dudz = np.zeros([len(erai_T.lat[:]),2])
rvalue_T_dudz = np.zeros([len(erai_T.lat[:]),2])
pres_p = 67
i_p=0
for i_lat in range(len(erai_T.lat[:])):
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(T_series(i_lat=i_lat, pres=pres_p).squeeze()[:], du_dz_series(lat=0, pres=pres_p).squeeze()[:])
    slope_T_dudz[i_lat,i_p] = slope
    rvalue_T_dudz[i_lat,i_p] = r_value
    
pres_p = 90
i_p=1
for i_lat in range(len(erai_T.lat[:])):
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(T_series(i_lat=i_lat, pres=pres_p).squeeze()[:], du_dz_series(lat=0, pres=pres_p).squeeze()[:])
    slope_T_dudz[i_lat,i_p] = slope
    rvalue_T_dudz[i_lat,i_p] = r_value

In [13]:
slope_var = pyg.Var([erai_lat, pyg.Pres([67,96])], values = slope_T_dudz, name='slope')
rvalue_var = pyg.Var([erai_lat, pyg.Pres([67,96])], values = rvalue_T_dudz, name='r value')

In [40]:
slope_int = np.zeros([np.int(np.ceil(len(erai_T.lat[:])/2)),2])
for i, lat in enumerate(slope_var.lat[:89]):
    print(i,lat)
    slope_int[i,:] = slope_var(lat=(-lat,lat)).integrate(pyg.Lat, dx=slope_var.lat(lat=(-lat,lat))*np.pi/180).squeeze()[-1,:]

0 89.0
1 87.99999999999999
2 87.0
3 86.0
4 85.0
5 84.0
6 83.00000000000001
7 82.0
8 81.0
9 80.0
10 79.0
11 78.0
12 77.00000000000001
13 76.0
14 75.00000000000001
15 74.0
16 73.0
17 72.0
18 71.0
19 70.0
20 68.99999999999999
21 68.0
22 66.99999999999999
23 66.0
24 65.0
25 64.0
26 63.0
27 62.0
28 61.0
29 59.99999999999999
30 59.0
31 58.0
32 57.0
33 56.0
34 55.0
35 54.0
36 53.0
37 52.00000000000001
38 51.0
39 50.0
40 49.0
41 48.0
42 47.0
43 46.0
44 45.0
45 43.99999999999999
46 43.0
47 42.0
48 41.0
49 40.0
50 39.0
51 38.0
52 37.0
53 36.0
54 35.0
55 34.0
56 33.0
57 32.0
58 31.0
59 29.999999999999996
60 29.0
61 28.0
62 27.0
63 26.000000000000004
64 25.0
65 24.0
66 23.0
67 21.999999999999996
68 21.0
69 20.0
70 19.0
71 18.0
72 17.0
73 16.0
74 14.999999999999998
75 14.0
76 13.000000000000002
77 12.0
78 10.999999999999998
79 10.0
80 9.0
81 8.0
82 7.0
83 6.0
84 5.0
85 4.0
86 3.0
87 2.0
88 1.0


In [46]:
lat=10
print(slope_var(lat=(-10,10))[:])
print(slope_var(lat=(-lat,lat)).integrate(pyg.Lat, dx=slope_var.lat(lat=(-lat,lat))*np.pi/180).squeeze()[:])

[[-5.52157412 -4.03627066]
 [-6.64846785 -4.78395647]
 [-7.50625535 -5.38409163]
 [-8.10865472 -5.85856976]
 [-8.49757965 -6.22627305]
 [-8.72509679 -6.50912922]
 [-8.84261218 -6.7235061 ]
 [-8.89288734 -6.87940765]
 [-8.90718672 -6.98526108]
 [-8.90639836 -7.04995549]
 [-8.90275735 -7.07819519]
 [-8.90078008 -7.07107456]
 [-8.89695838 -7.02777774]
 [-8.87815143 -6.94620996]
 [-8.82201705 -6.81684731]
 [-8.69331851 -6.62068644]
 [-8.44283368 -6.33461687]
 [-8.00949045 -5.92540308]
 [-7.32637757 -5.35505096]
 [-6.33975794 -4.58907933]
 [-5.03484633 -3.60317466]]
[[0.         0.        ]
 [0.10620365 0.076971  ]
 [0.22972691 0.16570396]
 [0.36599271 0.26381469]
 [0.51090944 0.36927484]
 [0.66120565 0.48041219]
 [0.81451283 0.59588872]
 [0.96928426 0.71459653]
 [1.12461921 0.83558859]
 [1.28007207 0.95806896]
 [1.43548627 1.08136034]
 [1.59085144 1.20483601]
 [1.74616601 1.3278717 ]
 [1.9012831  1.44981775]
 [2.05574621 1.56992308]
 [2.20859635 1.68718769]
 [2.35813749 1.80024404]
 [2.501

In [36]:
slope_int_var = pyg.Var([erai_lat(lat=(0,90)), pyg.Pres([67,96])], values = np.flip(slope_int,0), name='slope int')

In [147]:
fig, axes = plt.subplots()
ax_a = pyg.plot.AxesWrapper() 
ax_a = pyg.showvar(rvalue_var(pres=67), axes=ax_a, fig=fig, label='67 hPa')
ax_a = pyg.showvar(rvalue_var(pres=96), axes=ax_a, fig=fig, label='96 hPa')

ax_a.legend()
ax_a.render(fig=fig)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
fig, axes = plt.subplots()
ax_a = pyg.plot.AxesWrapper() 
ax_a = pyg.showvar(slope_int_var(pres=67), axes=ax_a, fig=fig, label='67 hPa')
#ax_a = pyg.showvar(rvalue_var(pres=96), axes=ax_a, fig=fig, label='96 hPa')

ax_a.legend()
ax_a.render(fig=fig)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …